In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings 
warnings.simplefilter('ignore')
import plotly.express as px
%matplotlib inline

In [2]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
def reading_taz_religion_destribution():
    religion_taz = pd.read_csv(r'/Users/yotamdery/Old Desktop/git/SEIR_model_covid _yotams/Data/raw/religion2taz.csv', index_col= 0)
    religion_taz = religion_taz.sort_values('taz_id')
    
    religion_taz['Generall'] = religion_taz['Other'] + religion_taz["Jewish"] + religion_taz['Christian']
    religion_taz['Arabs'] = religion_taz['Druze'] + religion_taz['Muslim']
    
    religion_taz = religion_taz[['Orth', 'Generall', 'Arabs']]
    religion_taz.rename(columns= {'Orth' : 'Orthodox'}, inplace= True)
    religion_taz.reset_index(inplace= True)
    return religion_taz

In [4]:
# A mapper - taz for 250 regions 
def reading_taz_250_mapping():
    taz_250_mapping = pd.read_excel(r'/Users/yotamdery/Old Desktop/git/health_data_mapping/Taz_Yeshuv_250_mapping.xlsx',engine='openpyxl', usecols= "A,P,AH")
    taz_250_mapping = taz_250_mapping.rename(columns= {'Unnamed: 0' : 'taz_id', 'Unnamed: 15' : 'population', 'Unnamed: 33' : '250_regions'})
    taz_250_mapping = taz_250_mapping.loc[3:2632]
    taz_250_mapping = taz_250_mapping[taz_250_mapping['population'] > 0]
    return taz_250_mapping

In [5]:
# Creating the religion destribution by 250 regions
def creating_religion_dist_to_250(religion_taz, taz_to_250_mapping):
    merged_df = religion_taz.merge(taz_to_250_mapping)
    religion_250 = merged_df.groupby('250_regions').mean().reset_index()

    return religion_250
    

In [6]:
def reading_250_30_mapping():
    mapping_30_250 = pd.read_excel(r'/Users/yotamdery/Old Desktop/git/Covid_data_investigation/cell250_to_cell30.xlsx',engine='openpyxl')
    return mapping_30_250

In [7]:
def creating_religion_dist_to_30(religion_250, mapping_30_250):
    merged_df = religion_250.merge(mapping_30_250, how= 'inner', left_on= '250_regions', right_on= 'cell_id')
    religion_30 = merged_df.groupby('30_county').mean().reset_index()
    return religion_30

In [8]:
def main():
    religion_taz = reading_taz_religion_destribution()
    taz_to_250_mapping = reading_taz_250_mapping()
    
    religion_250 = creating_religion_dist_to_250(religion_taz, taz_to_250_mapping)       # Creating the religion destribution by 250 regions
    
    mapping_30_250 = reading_250_30_mapping()
    religion_30 = creating_religion_dist_to_30(religion_250, mapping_30_250)        # Creating the religion destribution by 30 regions
    
    return religion_30[['30_county', 'Orthodox', 'Generall', 'Arabs']]
    
    

In [9]:
religion_destribution_30_counties = main()

In [12]:
religion_destribution_30_counties.to_csv('religion_destribution_30_counties.csv', index= False)